In [1]:
import duckdb

# start a DuckDB connection
con = duckdb.connect(database=':memory:')  # or 'mydb.duckdb' if you want persistence

In [15]:
# Replace with your actual paths

addr_root = "../data/ppd_addresses"

con.execute(f"""
    CREATE OR REPLACE VIEW ppd_src AS
    SELECT *
    FROM parquet_scan('{addr_root}/*.parquet', hive_partitioning=1)
""")

In [20]:
# Quick sanity check: expect only GL / 2025 rows below
con.execute("""
    SELECT *
    FROM ppd_src
    LIMIT 1
""").fetchdf()

,property_type,postcode,paon,saon,street,locality,town_city,district,county
0,S,AL35NA,41,None,GRANGE STREET,ST. ALBANS,ST. ALBANS,ST ALBANS,HERTFORDSHIRE


In [14]:
pp

,postcode
0,AL35NA
1,AL108TY


In [22]:
tdf = con.execute("""
    SELECT DISTINCT property_type, paon, saon
    FROM ppd_src
    WHERE paon IS NOT NULL
      AND TRIM(paon) <> ''
      AND LOWER(paon) LIKE '%flat%'
      AND property_type != 'F'
""").fetchdf()
tdf

,property_type,paon,saon
0,O,"FLATS 1-6, 56A",None
1,O,WESTVILLE FLATS,10
2,O,FLAT 3 DOLPHIN COURT,GARAGE ASSOCIATED WITH
3,O,MEMORIAL BUILDING FLATS,11
4,O,"FLAT 2D, 408",None
...,...,...,...
463,O,SOUTHLEIGH FLAT,None
464,O,ABBEY FLATTS,None
465,O,ST MARGARETS FLATS,None
466,O,SEMINARY FLATS,1


In [6]:
pp

,saon
0,None


- Review each column to identify patterns and create strategies for tokenization

In [9]:
pp

,saon
0,8
1,FLAT 4
2,FIRST FLOOR FLAT
3,16
4,FLAT 25
...,...
69021,35 EDISON HOUSE
69022,TY CRUG
69023,DRYSTONES
69024,FLAT 1-1A


In [12]:
from addr_utils.tokens import tokenize, tokens2features

In [13]:
tokenize("flat 25 willowbrook drive GL510pu cheltenham gloucestershire")

['FLAT', '25', 'WILLOWBROOK', 'DRIVE', 'GL510PU', 'CHELTENHAM']

In [14]:
tokens2features(['FLAT', '25', 'WILLOWBROOK', 'DRIVE', 'GL510PU', 'CHELTENHAM'])

[{'digits': 'no_digits',
  'word': 'FLAT',
  'length': 'w:4',
  'endsinpunc': False,
  'directional': False,
  'outcode': False,
  'posttown': False,
  'has.vowels': True,
  'flat': True,
  'company': False,
  'road': False,
  'residential': False,
  'business': False,
  'locational': False,
  'ordinal': False,
  'hyphenations': 0,
  'next': {'digits': 'all_digits',
   'word': False,
   'length': 'd:2',
   'endsinpunc': False,
   'directional': False,
   'outcode': False,
   'posttown': False,
   'has.vowels': False,
   'flat': False,
   'company': False,
   'road': False,
   'residential': False,
   'business': False,
   'locational': False,
   'ordinal': False,
   'hyphenations': 0},
  'rawstring.start': True},
 {'digits': 'all_digits',
  'word': False,
  'length': 'd:2',
  'endsinpunc': False,
  'directional': False,
  'outcode': False,
  'posttown': False,
  'has.vowels': False,
  'flat': False,
  'company': False,
  'road': False,
  'residential': False,
  'business': False,
  'lo